In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment
48              4                                   Rất tốt
76              5                                    Rất ok
107             2                             pin tụt nhanh
135             5                              sản phẩm tốt
173             2                             hao pin nhanh
177             5                              sản phẩm tốt
181             5                                    Rất ok
183             5                              sản phẩm tốt
192             5                               sản phẩm ok
205             5                               máy dùng ok
225             4                                       Tốt
227             5                               sản phẩm ok
237             5                               máy dùng ok
250             5                              sản phẩm tốt
426  

In [4]:
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

# Giả sử df là DataFrame của bạn và 'Comment' là cột chứa dữ liệu văn bản
df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment
0            4                      điện thoại này dùng rất thích
1            4                               sử dụng thấy cũng ok
2            2                       bảo hành ít quá chỉ 12 tháng
3            5                              sản phẩm mượt chạy êm
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...


In [10]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'tiêu cực'
    elif rating == 3:
        return 'trung tính'
    elif rating in [4, 5]:
        return 'tích cực'
    else:
        return 'không rõ'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                       bảo hành ít quá chỉ 12 tháng    tiêu cực
3            5                              sản phẩm mượt chạy êm    tích cực
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [5]:
#Tải bộ word2vec đã được train sẵn dành cho tiếng việt
import torch
import torchtext.vocab as vocab

word_embedding = vocab.Vectors(name = "vi_word2vec.txt",
                               unk_init = torch.Tensor.normal_)

word_embedding.vectors.shape

c:\Users\vuhamy\anaconda3\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\vuhamy\anaconda3\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
100%|██████████| 1587507/1587507 [03:57<00:00, 6674.38it/s] 


torch.Size([1587507, 100])

In [6]:
#Tạo hàm Get_Vector để truy xuất vector embedding từ bộ embedding đã tải
def get_vector(embeddings, word):
    """ Get embedding vector of the word
    @param embeddings (torchtext.vocab.vectors.Vectors)
    @param word (str)
    @return vector (torch.Tensor)
    """
    assert word in embeddings.stoi, f'*{word}* is not in the vocab!'
    return embeddings.vectors[embeddings.stoi[word]]

Kiểm tra từ có trong từ điển hay không: Hàm sử dụng một lệnh assert để kiểm tra xem từ word có tồn tại trong từ điển của embeddings hay không. Nếu không tồn tại, nó sẽ hiển thị một thông báo lỗi.

Trả về vector embedding: Nếu từ tồn tại trong từ điển, hàm sẽ sử dụng embeddings.stoi[word] để lấy chỉ số của từ trong danh sách các từ (từ điển), sau đó truy cập vào embeddings.vectors để lấy vector embedding tương ứng.

Đầu ra là Vector embedding tương ứng với từ word.

In [7]:
#Tạo hàm Get_Vector để tìm các từ gần nghĩa nhất với một từ cho trước, dựa trên mức độ tương tự trong không gian vector
def closest_words(embeddings, vector, n=10):
    """ Return n words closest in meaning to the word
    @param embeddings (torchtext.vocab.vectors.Vectors)
    @param vector (torch.Tensor)
    @param n (int)
    @return words (list(tuple(str, float)))
    """
    distances = [(word, torch.dist(vector, get_vector(embeddings, word)).item())
                 for word in embeddings.itos]

    return sorted(distances, key = lambda w: w[1])[:n]

Tính toán khoảng cách: Hàm sử dụng một list comprehension để duyệt qua tất cả các từ trong từ điển (embeddings.itos). Với mỗi từ, nó sẽ tính khoảng cách Euclid giữa vector vector cho trước và vector embedding của từ đó sử dụng hàm torch.dist.

Sắp xếp và trả về kết quả: Kết quả được sắp xếp theo khoảng cách tăng dần (key=lambda w: w[1]), và chỉ lấy n từ gần nghĩa nhất bằng cách sử dụng slicing [:n].

# Xây dựng từ điển

In [8]:
from itertools import chain
from collections import Counter

import torch
from tqdm import tqdm
from underthesea import word_tokenize

In [9]:
class Vocabulary:
    """ The Vocabulary class is used to record words, which are used to convert
        text to numbers and vice versa.
    """

    def __init__(self):
        self.word2id = dict()
        self.word2id['<pad>'] = 0   # Pad Token
        self.word2id['<unk>'] = 1   # Unknown Token
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return len(self.word2id)

    def id2word(self, word_index):
        """
        @param word_index (int)
        @return word (str)
        """
        return self.id2word[word_index]

    def add(self, word):
        """ Add word to vocabulary
        @param word (str)
        @return index (str): index of the word just added
        """
        if word not in self:
            word_index = self.word2id[word] = len(self.word2id)
            self.id2word[word_index] = word
            return word_index
        else:
            return self[word]

    @staticmethod
    def tokenize_corpus(corpus):
        """Split the documents of the corpus into words
        @param corpus (list(str)): list of documents
        @return tokenized_corpus (list(list(str))): list of words
        """
        print("Tokenize the corpus...")
        tokenized_corpus = list()
        for document in tqdm(corpus):
            tokenized_document = [word.replace(" ", "_") for word in word_tokenize(document)]
            tokenized_corpus.append(tokenized_document)

        return tokenized_corpus

    def corpus_to_tensor(self, corpus, is_tokenized=False):
        """ Convert corpus to a list of indices tensor
        @param corpus (list(str) if is_tokenized==False else list(list(str)))
        @param is_tokenized (bool)
        @return indicies_corpus (list(tensor))
        """
        if is_tokenized:
            tokenized_corpus = corpus
        else:
            tokenized_corpus = self.tokenize_corpus(corpus)
        indicies_corpus = list()
        for document in tqdm(tokenized_corpus):
            indicies_document = torch.tensor(list(map(lambda word: self[word], document)),
                                             dtype=torch.int64)
            indicies_corpus.append(indicies_document)

        return indicies_corpus

    def tensor_to_corpus(self, tensor):
        """ Convert list of indices tensor to a list of tokenized documents
        @param indicies_corpus (list(tensor))
        @return corpus (list(list(str)))
        """
        corpus = list()
        for indicies in tqdm(tensor):
            document = list(map(lambda index: self.id2word[index.item()], indicies))
            corpus.append(document)

        return corpus

# Với bộ dữ liệu 

In [14]:
class TGDD():
    """ Load dataset from file csv"""

    def __init__(self, vocab, csv_fpath='product_df.csv', tokenized_fpath=None):
        """
        @param vocab (Vocabulary)
        @param csv_fpath (str)
        @param tokenized_fpath (str)
        """
        self.vocab = vocab
        self.pad_idx = vocab["<pad>"]
        df = pd.read_csv(csv_fpath)
        self.sentiments_list = list(df.sentiment)
        self.reviews_list = list(df.vi_review)

        sentiments_type = list(set(self.sentiments_list))
        sentiments_type.sort()

        self.sentiment2id = {sentiment: i for i, sentiment in enumerate(sentiments_type)}

        if tokenized_fpath:
            self.tokenized_reviews = torch.load(tokenized_fpath)
        else:
            self.tokenized_reviews = self.vocab.tokenize_corpus(self.reviews_list)

        self.tensor_data = self.vocab.corpus_to_tensor(self.tokenized_reviews, is_tokenized=True)
        self.tensor_label = torch.tensor([self.sentiment2id[sentiment] for sentiment in self.sentiments_list],
                                         dtype=torch.float64)

    def __len__(self):
        return len(self.tensor_data)

    def __getitem__(self, idx):
        return self.tensor_data[idx], self.tensor_label[idx]

    def collate_fn(self, examples):
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)

        reviews = [e[0] for e in examples]
        reviews = torch.nn.utils.rnn.pad_sequence(reviews,
                                                  batch_first=False,
                                                  padding_value=self.pad_idx)
        reviews_lengths = torch.tensor([len(e[0]) for e in examples])
        sentiments = torch.tensor([e[1] for e in examples])

        return {"Comment": (reviews, reviews_lengths), "Sentiment": sentiments}

In [15]:
dataset = TGDD(vocab=vocab, csv_fpath="product_df.csv", tokenized_fpath="tokenized.pt")

TypeError: 'module' object is not subscriptable